In [1]:
from tensorflow import keras
import numpy as np
import os
from seismic_purifier.config import BATCH_SIZE
from seismic_purifier.representation_learning_models import (
    RepresentationLearningSingleAutoencoder,
    RepresentationLearningDenoisingSingleAutoencoder,
    RepresentationLearningMultipleAutoencoder
)

2025-03-19 00:56:54.485041: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 00:56:54.514668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-19 00:56:54.514691: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-19 00:56:54.514708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 00:56:54.520761: I tensorflow/core/platform/cpu_feature_g

In [2]:
# ============================
# 1. Configuration
# ============================

# Paths to your data
TRAIN_DATA_PATH = "/home/ege/rawcovar/ADVT_SUBSET_NPY/./KO.ADVT..HHN__20190831T235957120000Z__20190902T000002810000Z_processed.npy"#'data/X_train_1280sample.npy'  # Replace with your actual path
TEST_DATA_PATH = 'data/X_test_1280sample.npy'  # Replace with your actual path
TRAIN_LABEL_PATH = 'data/Y_train_1280sample.npy'  # Replace with your actual path
TEST_LABEL_PATH = 'data/Y_test_1280sample.npy'  # Replace with your actual path

# Directory to save checkpoints and the final model
CHECKPOINT_DIR = 'checkpoints'
MODEL_SAVE_PATH = 'checkpoints/representation_cross_covariances.h5'

# Training parameters
EPOCHS = 5
LEARNING_RATE = 1e-3

In [3]:
# Ensure directories exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# ============================
# 2. Data loading
# ============================
X_train = np.load(TRAIN_DATA_PATH)  # Expected shape: (num_samples, 3000, 3)
print(f"Training data shape: {X_train.shape}")

Training data shape: (8640, 3000, 3)


In [4]:
# ============================
# 3. Representation Learning Model Instantiation
# ============================

# Choose the model you want to train
# For example, using RepresentationLearningSingleAutoencoder
#model = RepresentationLearningSingleAutoencoder(
#    name="rep_learning_autoencoder"
#)

# Alternatively, you can choose other models:
# model = RepresentationLearningDenoisingSingleAutoencoder(
#     name="rep_learning_denoising_autoencoder",
#     input_noise_std=1e-6,
#     denoising_noise_std=2e-1
# )
model = RepresentationLearningMultipleAutoencoder(
     name="rep_learning_autoencoder_ensemble",
     input_noise_std=1e-6,
     eps=1e-27
)

2025-03-19 00:57:01.095304: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22286 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2025-03-19 00:57:01.096367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22286 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2025-03-19 00:57:01.097269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 20267 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:67:00.0, compute capability: 8.6
2025-03-19 00:57:01.098195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 22270 MB memory:  -> device: 3, name: NVIDIA GeForce RTX 3090

In [5]:
model.name

'rep_learning_autoencoder_ensemble'

In [6]:
# ============================
# 4. Model Compilation
# ============================
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE) 
model.compile(optimizer=optimizer)

In [7]:
# ============================
# 5. Callbacks Setup
# ============================
# Define callbacks for saving checkpoints, early stopping.
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(CHECKPOINT_DIR, 'autoencoder_epoch_{epoch:02d}.h5'),
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True,
        verbose=1
    )
]

In [8]:
# ============================
# 6. Training the Representation Learning Model and Save.
# ============================
fit_result = model.fit(X_train, 
                       epochs=EPOCHS, 
                       batch_size=BATCH_SIZE, 
                       callbacks=callbacks, 
                       shuffle=False)

model.save_weights(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

Epoch 1/5


2025-03-19 00:59:00.573693: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-03-19 00:59:01.750725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2025-03-19 00:59:02.296978: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-03-19 00:59:02.895814: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f88c800b520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-19 00:59:02.895855: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-03-19 00:59:02.895867: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-03-19 00:59:02.895877: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor de

34/34 [==============================] - ETA: 0s - loss: 2.4856
Epoch 1: saving model to checkpoints/autoencoder_epoch_01.h5
34/34 [==============================] - 140s 546ms/step - loss: 2.4856
Epoch 2/5
34/34 [==============================] - ETA: 0s - loss: 2.2234
Epoch 2: saving model to checkpoints/autoencoder_epoch_02.h5
34/34 [==============================] - 14s 418ms/step - loss: 2.2234
Epoch 3/5
34/34 [==============================] - ETA: 0s - loss: 1.9848
Epoch 3: saving model to checkpoints/autoencoder_epoch_03.h5
34/34 [==============================] - 15s 434ms/step - loss: 1.9848
Epoch 4/5
34/34 [==============================] - ETA: 0s - loss: 1.7839
Epoch 4: saving model to checkpoints/autoencoder_epoch_04.h5
34/34 [==============================] - 15s 433ms/step - loss: 1.7839
Epoch 5/5
34/34 [==============================] - ETA: 0s - loss: 1.6541
Epoch 5: saving model to checkpoints/autoencoder_epoch_05.h5
34/34 [==============================] - 15s 431ms